In [1]:
import wfdb
import scipy
import os
import glob
import subprocess

In [2]:

DATA_ROOT = 'D:/ECG/processed/physionet2021small'
PHYSIONET_PATH = 'D:/ECG/physionet.org/files/challenge-2021/1.0.3/training_small'

In [3]:
MANIFEST_PATH = 'D:/ECG/processed/physionet2021small/manifests'
FINE_TUNE_MANIFEST = f'{MANIFEST_PATH}/cinc'
TOTAL_MANIFEST = f'{MANIFEST_PATH}/total/train.tsv'
PRE_TRAINING_MANIFEST = f"{MANIFEST_PATH}/cmsc"

In [4]:
FAIRSEQ_SIG_DIR = 'C:/Users/david/Documents/PythonScripts/fairseq-signals'
CONVERT_TO_CLOCS_MANIFEST = f'{FAIRSEQ_SIG_DIR}/fairseq_signals/data/ecg/preprocess/convert_to_clocs_manifest.py'
PRE_TRAINING_CONFIG = f'{FAIRSEQ_SIG_DIR}/examples/w2v_cmsc/config/pretraining'
FINE_TUNING_CONFIG = f'{FAIRSEQ_SIG_DIR}/examples/w2v_cmsc/config/finetuning/ecg_transformer'

In [36]:
cmd = f"python {FAIRSEQ_SIG_DIR}/fairseq_signals/data/ecg/preprocess/preprocess_physionet2021.py \
        {PHYSIONET_PATH} \
        --dest {DATA_ROOT} \
        --subset WFDB_ChapmanShaoxing\
        --workers 1"

In [37]:
result = subprocess.run(cmd, stdout=subprocess.PIPE, shell=True)
print(result.stdout.decode('utf-8'))

In [38]:
cmd = f"python {FAIRSEQ_SIG_DIR}/fairseq_signals/data/ecg/preprocess/manifest.py \
        {DATA_ROOT} \
        --dest {MANIFEST_PATH} \
        --subset ChapmanShaoxing \
        --combine_subsets ChapmanShaoxing"

In [39]:
result = subprocess.run(cmd, stdout=subprocess.PIPE, shell=True)
print(result.stdout.decode('utf-8'))

In [40]:
cmd = f"python {CONVERT_TO_CLOCS_MANIFEST} \
            {TOTAL_MANIFEST} \
            --dest {MANIFEST_PATH}"
print(cmd)

python C:/Users/david/Documents/PythonScripts/fairseq-signals/fairseq_signals/data/ecg/preprocess/convert_to_clocs_manifest.py             D:/ECG/processed/physionet2021small/manifests/total/train.tsv             --dest D:/ECG/processed/physionet2021small/manifests


In [41]:
result = subprocess.run(cmd, stdout=subprocess.PIPE, shell=True)
print(result.stdout.decode('utf-8'))

In [42]:
cmd = f"fairseq-hydra-train \
            task.data={PRE_TRAINING_MANIFEST} \
            --config-dir {PRE_TRAINING_CONFIG} \
            --config-name w2v_cmsc_rlm_small"

In [ ]:
result = subprocess.run(cmd, stdout=subprocess.PIPE, shell=True)
print(result.stdout.decode('utf-8'))

In [11]:
PRETRAIN_CHECKPOINT = f'C:/Users/david/Documents/PythonScripts/ECGPretraining/outputs/2023-04-15/13-50-53/checkpoints/checkpoint100.pt'
WEIGHTS_PATH = f'{PHYSIONET_PATH}/weights.csv'

In [17]:
cmd = f"fairseq-hydra-train \
        task.data={FINE_TUNE_MANIFEST} \
        model.model_path={PRETRAIN_CHECKPOINT} \
        --config-dir {FINE_TUNING_CONFIG} \
        --config-name diagnosis"

In [18]:
result = subprocess.run(cmd, stdout=subprocess.PIPE, shell=True)
print(result.stdout.decode('utf-8'))

[2023-04-15 17:01:46,493][fairseq_cli.train][INFO] - {'_name': None,
 'checkpoint': {'_name': None, 'save_dir': 'checkpoints', 'restore_file': 'checkpoint_last.pt', 'finetune_from_model': None, 'reset_dataloader': False, 'reset_lr_scheduler': False, 'reset_meters': False, 'reset_optimizer': False, 'optimizer_overrides': '{}', 'save_interval': 1, 'save_interval_updates': 0, 'keep_interval_updates': -1, 'keep_interval_updates_pattern': -1, 'keep_last_epochs': 1, 'keep_best_checkpoints': -1, 'no_save': False, 'no_epoch_checkpoints': False, 'no_last_checkpoints': False, 'no_save_optimizer_state': False, 'best_checkpoint_metric': 'loss', 'maximize_best_checkpoint_metric': False, 'patience': -1, 'checkpoint_suffix': '', 'checkpoint_shard_count': 1, 'load_checkpoint_on_all_dp_ranks': False},
 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 10, 'log_format': 'json', 'log_file': None, 'wandb_project': None, 'wandb_entity': None, 'seed': 1, 'fp16': False, 'memory_efficient_fp

In [61]:
import torch
torch.cuda.empty_cache()
import gc
gc.collect()

248

In [62]:
torch.cuda.memory_summary(device=None, abbreviated=False)

'|===========================================================================|\n|                  PyTorch CUDA memory summary, device ID 0                 |\n|---------------------------------------------------------------------------|\n|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |\n|===========================================================================|\n|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |\n|---------------------------------------------------------------------------|\n| Allocated memory      |      0 B   |      0 B   |      0 B   |      0 B   |\n|       from large pool |      0 B   |      0 B   |      0 B   |      0 B   |\n|       from small pool |      0 B   |      0 B   |      0 B   |      0 B   |\n|---------------------------------------------------------------------------|\n| Active memory         |      0 B   |      0 B   |      0 B   |      0 B   |\n|       from large pool |      0 B   |      0 B   |